In [1]:
import numpy as np
import sys
import os
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
os.chdir("/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data")

In [2]:
!ls
!pwd

1.csv			 Lung12  Lung5_Rep1  Lung5_Rep3  Lung9_Rep1  vis   vis2
cosMX_showcase_meta.csv  Lung13  Lung5_Rep2  Lung6	 Lung9_Rep2  vis1  vis3


/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data


In [3]:
gene_expression = pd.read_csv('./Lung5_Rep2/Lung5_Rep2-Flat_files_and_images/Lung5_Rep2_exprMat_file.csv')
cell_boundary = pd.read_csv('./Lung5_Rep2/Lung5_Rep2-Flat_files_and_images/Lung5_Rep2_metadata_file.csv')
fov_position = pd.read_csv('./Lung5_Rep2/Lung5_Rep2-Flat_files_and_images/Lung5_Rep2_fov_positions_file.csv')


In [4]:
x_coordinate = [x / 10000 for x in list(fov_position["x_global_px"])]
y_coordinate = [y / 10000 for y in list(fov_position["y_global_px"])]


In [5]:
"""
CosMx: 
1. All FOVs are the same dimension, 5472 x 3648 pixels
2. multiply the pixel value by 0.18 um per pixel
3. FOV area: 5472 x 3648 pixels -> 984.96um x 656.64um = 646,764.134 um2 


New Benchamrk from CosMx
1. length: 5472 pixels, width: 3648 pixels
2. simulated spot: 
    length: 5472 pixels / 5 = 1094.4 pixel = 196.992 um
    width: 3648 pixels / 4 = 912 pixel = 164.16 um
    one spot area: 196.992 um * 164.16 um = 32338.2067 um2
3. In total: 20 spots / FOV

spot_fov_cellId_mapping.csv (spot_id, fov, cell_Id)

spot_gene_expression.csv (spot_id, gene1, gene2, gene3...)

"""

'\nCosMx: \n1. All FOVs are the same dimension, 5472 x 3648 pixels\n2. multiply the pixel value by 0.18 um per pixel\n3. FOV area: 5472 x 3648 pixels -> 984.96um x 656.64um = 646,764.134 um2 \n\n\nNew Benchamrk from CosMx\n1. length: 5472 pixels, width: 3648 pixels\n2. simulated spot: \n    length: 5472 pixels / 5 = 1094.4 pixel = 196.992 um\n    width: 3648 pixels / 4 = 912 pixel = 164.16 um\n    one spot area: 196.992 um * 164.16 um = 32338.2067 um2\n3. In total: 20 spots / FOV\n\nspot_fov_cellId_mapping.csv (spot_id, fov, cell_Id)\n\nspot_gene_expression.csv (spot_id, gene1, gene2, gene3...)\n\n'

In [6]:
def get_spot_fov_cellId_mapping(data_result, cell_boundary_fov_11):
    new_col_val = cell_boundary_fov_11.shape[0]* [0]
    cell_boundary_fov_11.insert(loc=0, column='spot_id', value=new_col_val)

    for i in range(cell_boundary_fov_11.shape[0]):
        one_row_sample = cell_boundary_fov_11.iloc[i]

        center_x = one_row_sample["CenterX_local_px"]
        center_y = one_row_sample["CenterY_local_px"]

        # Calculate spot_id based on CenterX_local_px and CenterY_local_px values
        spot_id = 1 + 4 * int((center_x - 1e-6) / 1094.4) + int((center_y - 1e-6) / 912)

        # Use spot_id as needed in your code
        # print(spot_id)
        data_result = data_result.append({'spot_id' : spot_id, 'fov' : one_row_sample["fov"], 'cell_ID' : one_row_sample["cell_ID"]}, ignore_index = True)
    return data_result
    
data_final_result = pd.DataFrame(columns = ['spot_id', 'fov', 'cell_ID'])
fov_ids_lst = cell_boundary['fov'].unique()
print("fov_ids_lst:", fov_ids_lst)

for fov_id in tqdm(fov_ids_lst):
    # print("fov_id:", fov_id)
    cell_boundary_fov =  cell_boundary[(cell_boundary['fov']==fov_ids_lst[fov_id-1])]
    # print("cell_boundary_fov_without_spot_id:", cell_boundary_fov.shape)
    data_final_result = get_spot_fov_cellId_mapping(data_final_result, cell_boundary_fov)


fov_ids_lst: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


100%|██████████| 30/30 [01:36<00:00,  3.21s/it]


In [7]:
spot_celld_mapping = data_final_result

In [8]:
names = ['fov'] + ['spot-id=' + str(i) for i in range(1,21)]
fov_dic = {}
for i in names:
    fov_dic[i] = 0
    
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

fov_spot_cells_stats = pd.DataFrame(columns = names)


for fov_id in fov_ids_lst:
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    
    fov_dic_sample = fov_dic
    fov_dic_sample["fov"] = fov_id
    
    for i in spot_id_lst:
        spot_id_data = fov_data[(fov_data['spot_id']==i)]
        spot_id_num = "spot-id=" + str(i)
        fov_dic_sample[spot_id_num] = spot_id_data.shape[0]
    fov_spot_cells_stats = fov_spot_cells_stats.append(fov_dic_sample, ignore_index = True)



In [9]:
spot_gene_expression = ["fov", "spot_id"]
genes_name_lst = list(gene_expression.columns)[2:]
spot_gene_expression = spot_gene_expression + genes_name_lst
spot_gene_expression = pd.DataFrame(columns = spot_gene_expression)


In [10]:
def get_spot_gene_expression(spot_cell_mapping, fov_expression, spot_id):
    genes_lst = list(fov_expression.columns)[2:]
    
    cell_id_lst = spot_cell_mapping[(spot_cell_mapping['spot_id']==spot_id)]["cell_ID"].tolist()
    
    cell_gene_expression_total = len(genes_lst)*[0]
    for cell_id in cell_id_lst:
        cell_gene_expression = fov_expression[(fov_expression['cell_ID'] == cell_id)]
        cell_gene_expression = cell_gene_expression.values.tolist()[0][2:]
        cell_gene_expression_total = np.sum([cell_gene_expression_total, cell_gene_expression], axis=0).tolist()
    
    return cell_gene_expression_total
    

In [11]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

for fov_id in tqdm(fov_ids_lst):
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    fov_gene_expression = gene_expression[(gene_expression['fov'] == fov_id)]
    
    for spot_id in spot_id_lst:
        to_append = [fov_id, spot_id]
        spot_gene_express = get_spot_gene_expression(fov_data, fov_gene_expression, spot_id)
        to_append = to_append + spot_gene_express
        a_series = pd.Series(to_append, index = spot_gene_expression.columns)
        spot_gene_expression = spot_gene_expression.append(a_series, ignore_index=True)    
  

100%|██████████| 30/30 [00:43<00:00,  1.44s/it]


In [12]:
CosMx_cell_type = pd.read_csv('./cosMX_showcase_meta.csv')
sample_cell_count_dic = {}
sample_fov_count_dic = {}
for i in range(1, 9):
    sample_cell_count_dic[i] = 0
    sample_fov_count_dic[i] = 0
# print("cell:", sample_cell_count_dic)
# print("fov:", sample_fov_count_dic)

sample_2_index_lst = []

cell_id_lst = CosMx_cell_type["cell_ID"].tolist()
# print("len of cell_id_lst:", len(cell_id_lst))

fov_lst = []

index = 0
for i in range(len(cell_id_lst)):
    sample_id = int(cell_id_lst[i].split("_")[1])
    fov_id = int(cell_id_lst[i].split("_")[2])
    cell_id = int(cell_id_lst[i].split("_")[3])

    if sample_id == 2:
        sample_2_index_lst.append(index)
        
    sample_cell_count_dic[sample_id] =  sample_cell_count_dic[sample_id] + 1
    if sample_id == 2:
        if fov_id not in fov_lst:
            fov_lst.append(fov_id)
    index += 1

print(sample_cell_count_dic)
print(fov_lst)
print(len(sample_2_index_lst))


{1: 98002, 2: 105800, 3: 97809, 4: 89975, 5: 87606, 6: 139504, 7: 71304, 8: 81236}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
105800


In [13]:
CosMx_cell_type_sample_2 = CosMx_cell_type.iloc[sample_2_index_lst]
CosMx_cell_type_sample_2.reset_index(drop=True, inplace=True)
CosMx_cell_type_sample_2

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,98003,c_2_1_1,1,2142,1.48,59,40,1075,1842,22954,...,980p,74,118,2.071882,0.025428,False,Lung5,tumor 5,tumor-stroma boundary,0.56
1,98004,c_2_1_3,1,4845,1.15,94,82,4367,11890,1593,...,980p,158,283,2.451786,0.060983,False,Lung5,mast,plasmablast-enriched stroma,0.01
2,98005,c_2_1_4,1,2915,1.00,59,59,6017,13814,12387,...,980p,63,100,2.000000,0.021549,False,Lung5,neutrophil,stroma,0.02
3,98006,c_2_1_5,1,8634,1.02,106,104,3945,13832,10624,...,980p,106,188,2.274158,0.040512,False,Lung5,tumor 12,stroma,0.02
4,98007,c_2_1_6,1,2248,0.66,41,62,1918,5078,7462,...,980p,57,86,1.934498,0.018532,False,Lung5,tumor 13,plasmablast-enriched stroma,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105795,203798,c_2_30_4161,30,2051,1.92,71,37,399,947,124,...,980p,85,152,2.181844,0.032754,False,Lung5,fibroblast,stroma,0.00
105796,203799,c_2_30_4162,30,1538,1.67,55,33,832,2481,178,...,980p,73,121,2.082785,0.026074,False,Lung5,fibroblast,stroma,0.00
105797,203800,c_2_30_4163,30,1462,2.07,60,29,855,2047,152,...,980p,22,25,1.397940,0.005387,False,Lung5,fibroblast,stroma,0.00
105798,203801,c_2_30_4164,30,747,1.82,40,22,1061,1738,139,...,980p,17,27,1.431364,0.005818,False,Lung5,plasmablast,stroma,0.00


In [14]:

for i in CosMx_cell_type_sample_2.index:
    CosMx_cell_type_sample_2.at[i,'cell_ID']= int(CosMx_cell_type_sample_2.iloc[i]["cell_ID"].split("_")[3])

CosMx_cell_type_sample_2

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,98003,1,1,2142,1.48,59,40,1075,1842,22954,...,980p,74,118,2.071882,0.025428,False,Lung5,tumor 5,tumor-stroma boundary,0.56
1,98004,3,1,4845,1.15,94,82,4367,11890,1593,...,980p,158,283,2.451786,0.060983,False,Lung5,mast,plasmablast-enriched stroma,0.01
2,98005,4,1,2915,1.00,59,59,6017,13814,12387,...,980p,63,100,2.000000,0.021549,False,Lung5,neutrophil,stroma,0.02
3,98006,5,1,8634,1.02,106,104,3945,13832,10624,...,980p,106,188,2.274158,0.040512,False,Lung5,tumor 12,stroma,0.02
4,98007,6,1,2248,0.66,41,62,1918,5078,7462,...,980p,57,86,1.934498,0.018532,False,Lung5,tumor 13,plasmablast-enriched stroma,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105795,203798,4161,30,2051,1.92,71,37,399,947,124,...,980p,85,152,2.181844,0.032754,False,Lung5,fibroblast,stroma,0.00
105796,203799,4162,30,1538,1.67,55,33,832,2481,178,...,980p,73,121,2.082785,0.026074,False,Lung5,fibroblast,stroma,0.00
105797,203800,4163,30,1462,2.07,60,29,855,2047,152,...,980p,22,25,1.397940,0.005387,False,Lung5,fibroblast,stroma,0.00
105798,203801,4164,30,747,1.82,40,22,1061,1738,139,...,980p,17,27,1.431364,0.005818,False,Lung5,plasmablast,stroma,0.00


In [15]:
sample_2_dic = {}
for key in CosMx_cell_type_sample_2["cell_type"].tolist():
    if key not in sample_2_dic:
        sample_2_dic[key] = 1
    else:
        sample_2_dic[key] = sample_2_dic[key] + 1
# sample_2_dic


In [16]:
cell_type_lst = list(CosMx_cell_type_sample_2['cell_type'].unique())
print(len(cell_type_lst))
cell_type_lst

22


['tumor 5',
 'mast',
 'neutrophil',
 'tumor 12',
 'tumor 13',
 'endothelial',
 'plasmablast',
 'pDC',
 'fibroblast',
 'T CD4 naive',
 'epithelial',
 'T CD8 naive',
 'mDC',
 'macrophage',
 'B-cell',
 'NK',
 'monocyte',
 'T CD8 memory',
 'Treg',
 'T CD4 memory',
 'tumor 9',
 'tumor 6']

In [17]:
cell_id_lst = CosMx_cell_type_sample_2["cell_ID"].tolist()
len(cell_id_lst)

105800

In [18]:
def get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type, cell_type_dic):
    existig_cell_ids_with_celltype = CosMx_cell_type['cell_ID'].unique()
    
    for cell_id in one_spot_cell_lst:
        if cell_id in existig_cell_ids_with_celltype:
            one_cell_sample = CosMx_cell_type[(CosMx_cell_type['cell_ID']==cell_id)]
            cell_type = one_cell_sample["cell_type"].values[0]
            cell_type_dic[cell_type] = cell_type_dic[cell_type] + 1
        else:
            print("cell id not found in groud truth!!!")
        
    return cell_type_dic


In [19]:
fov_ids_lst = spot_celld_mapping['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

column_name_lst = ['fov', 'spot_id'] + cell_type_lst
ground_truth_table = pd.DataFrame(columns = column_name_lst)

for fov_id in fov_ids_lst:
    fov_data = spot_celld_mapping[(spot_celld_mapping['fov']==fov_id)]

    for spot_id in spot_id_lst:
        sample_dic = {}
        for i in column_name_lst:
            sample_dic[i] = 0
        
        spot_id_data = fov_data[(fov_data['spot_id']==spot_id)]
        one_spot_cell_lst = list(spot_id_data['cell_ID'].unique()) # all cell ids for one specific spot
        
        CosMx_cell_type_sample_fov = CosMx_cell_type_sample_2[(CosMx_cell_type_sample_2['fov']==fov_id)]
        sample_dic = get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type_sample_fov, sample_dic)
        
        sample_dic["fov"] = fov_id
        sample_dic["spot_id"] = spot_id
        ground_truth_table = ground_truth_table.append(sample_dic, ignore_index = True)




cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud t

In [20]:
ground_truth_table.rename(columns = {'tumor 5':'tumor'}, inplace = True)
del ground_truth_table['tumor 12']
del ground_truth_table['tumor 13']
del ground_truth_table['tumor 6']
del ground_truth_table['tumor 9']

In [21]:
def get_spot_x_y_range(x_global_px, y_global_px, fov_id, fov_spot_coordinates):
    fov_length = 5472
    fov_width = 3648
    x_l = x_global_px
    y_l = y_global_px
    x_h = x_l + fov_length
    y_h = y_l + fov_width
    
    for i in range(1, 6):  # 5
        for j in range(1, 5):  # 4
            spot_id = (i - 1) * 4 + j
            x = x_l + (5472 / 10.0) * 1 + (5472 / 5.0) * (i - 1)
            y = y_h - (3648 / 8.0) * 1 - (3648 / 4.0) * (j - 1)
            fov_spot_coordinates = fov_spot_coordinates.append(
                {'fov': fov_id, 'spot_id': spot_id, 'x': x, 'y': y},
                ignore_index=True
            )

    fov_spot_coordinates['x'] = fov_spot_coordinates['x']
    fov_spot_coordinates['y'] = fov_spot_coordinates['y']

    # fov_spot_coordinates['x'] = fov_spot_coordinates['x'] * 0.18 * 1e-4
    # fov_spot_coordinates['y'] = fov_spot_coordinates['y'] * 0.18 * 1e-4

    return fov_spot_coordinates



In [22]:
fov_spot_coordinates = pd.DataFrame(columns = ['fov', 'spot_id', 'x', 'y'])
for fov_id in fov_lst:
    x_px = fov_position[fov_position['fov']==fov_id]["x_global_px"].values[0]
    y_px = fov_position[fov_position['fov']==fov_id]["y_global_px"].values[0]
    fov_spot_coordinates = get_spot_x_y_range(x_px, y_px, fov_id, fov_spot_coordinates)
fov_spot_coordinates

,fov,spot_id,x,y
0,1.0,1.0,7897.200000,162364.222222
1,1.0,2.0,7897.200000,161452.222222
2,1.0,3.0,7897.200000,160540.222222
3,1.0,4.0,7897.200000,159628.222222
4,1.0,5.0,8991.600000,162364.222222
...,...,...,...,...
595,30.0,16.0,33069.288889,177878.222222
596,30.0,17.0,34163.688889,180614.222222
597,30.0,18.0,34163.688889,179702.222222
598,30.0,19.0,34163.688889,178790.222222


In [22]:
spot_gene_expression = spot_gene_expression.drop(columns = ["fov", "spot_id"])
ground_truth_table = ground_truth_table.drop(columns = ["fov", "spot_id"])
fov_spot_coordinates = fov_spot_coordinates.drop(columns = ["fov", "spot_id"])



import anndata as ad
st_adata = ad.AnnData(X = spot_gene_expression.values, obs = ground_truth_table, var = pd.DataFrame(index = list(spot_gene_expression.columns)), dtype=int)
st_adata.obsm["spatial"] = fov_spot_coordinates.values

spot_sums = np.sum(st_adata.X, axis=1)
mask = spot_sums > 100
filtered_data = st_adata[mask]

file_path = "/home/luqiaolin/projects/Benchmarking_paper_code/pseudo_spot_generation/cosmx_lung/Lung5_2.h5ad"
filtered_data.write_h5ad(file_path)



